# Chapter 11: Linear Constrained Optimization

In [1]:
from dataclasses import dataclass
import numpy as np

## Algorithm 11.1

In [2]:
@dataclass
class LinearProgram:
    A : np.ndarray
    b: np.ndarray
    c: np.ndarray

def get_vertex(B, LP):
    A, b, c = LP.A, LP.b, LP.c
    b_inds = np.sort(B)
    AB = A[:, b_inds]
    xB = np.linalg.solve(AB, b)
    x = np.zeros(len(c))
    x[b_inds] = xB
    return x

## Algorithm 11.2

In [38]:
def edge_transition(LP, B, q):
    A, b, c = LP.A, LP.b, LP.c
    n = A.shape[1]
    b_inds = np.sort(np.array(B)-1)
    n_inds = np.sort(np.setdiff1d(np.arange(0,n,1), np.array(B)-1))
    AB = A[:, b_inds]
    d, xB = np.linalg.solve(AB, A[:, n_inds[q-1]]), np.linalg.solve(AB, b)
 
    p, xq_prime = 0, np.infty

    for i in range(len(d)):
        if d[i] > 0:
            v = xB[i,:]/d[i]
            if v < xq_prime:
                p, xq_prime = i, v
    
    return p, xq_prime

### Example

In [39]:
A= np.array([
    [1, 1, 1, 0],
    [-4, 2, 0, 1]
])
b = np.array([[9], [2]])
c = np.array([[3], [-1], [0], [0]])

problema = LinearProgram(A, b, c)
B = [3,4] #base variables indexes (index start from 1)
q = 2 # entering variable (index start from 1) 
p, xp = edge_transition(problema, B, q)
B[p] = q

print(f"New Base: {B} --- Value of entering variable: {xp[0]}")

New Base: [3, 2] --- Value of entering variable: 1.0


## Algorithm 11.3

In [ ]:
def step_lp(B, LP):
    A, b, c = LP.A, LP.b, LP.c
    n = A.shape[1]
    b_inds = np.sort(np.array(B)-1)
    n_inds = np.sort(np.setdiff1d(np.arange(0,n,1), np.array(B)-1))
    AB, AV = A[:, b_inds], A[:, n_inds]
    xB = np.linalg.solve(AB, b)
    cB = c[:,b_inds]
    lambda_ = np.linalg.solve(AB.T, cB)
    cV = c[n_inds]
    muV = cV - np.matmul(AV.T, lambda_)

    q, p, xq_prime, delta = -1, 0, np.infty, np.infty
    for i  in range(len(muV)):
        if muV[i] < 0:
            pi, xi_prime = edge_transition(LP, B, i+1)
            if np.matmul(muV[i],xi_prime) < delta:
                q, p, xq_prime, delta = i, pi, xi_prime, np.matmul(muV[i],xi_prime)
    if q == -1:
        return B, True
    
    if xq_prime == np.infty:
        raise ValueError("unbounded")

    j = np.argmax(B == b_inds[p])
    B[j] = n_inds[q]
    return B, False